In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import train_test_split

In [15]:
set_train = pd.read_csv('data_set.csv')
test_values = pd.read_csv('data_test.csv',index_col='building_id')

In [16]:
features = ['age',
            'geo_level_1_id',
            'area_percentage',
            'count_floors_pre_eq',
            'geo_level_2_id',
            'height_percentage',
            'resistance',
            'base_condition']
train_values_subset = set_train[features[0:-1]]
train_labels = set_train.loc[:, 'damage_grade']

In [20]:
train_values_subset = pd.get_dummies(train_values_subset)

In [21]:
param_grid = {'randomforestclassifier__n_estimators': [100, 200, 300],
              'randomforestclassifier__max_depth': [30, 40, 50],
             }

In [22]:
pipe = make_pipeline(StandardScaler(), 
                    RandomForestClassifier(random_state=1))

In [23]:
rf_model = GridSearchCV(pipe, param_grid, cv=5)

In [24]:
rf_model.fit(train_values_subset, train_labels.values.ravel())

D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Anaconda\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "D:\Anaconda\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "D:\Anaconda\lib\site-packages\joblib\parallel.py", line 1044, in __call__
    while self.dispatch_one_batch(iterator):
  File "D:\Anaconda\lib\site-packages\joblib\parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "D:\Anaconda\lib\site-packages\joblib\parallel.py", line 777, in _d

D:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.68914645 0.69002917 0.69075069 0.68710854 0.68811406        nan
 0.68710854 0.68825223        nan]
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(random_state=1))]),
             param_grid={'randomforestclassifier__max_depth': [30, 40, 50],
                         'randomforestclassifier__n_estimators': [100, 200,
                                                                  300]})

In [25]:
rf_model.best_params_

{'randomforestclassifier__max_depth': 30,
 'randomforestclassifier__n_estimators': 300}

In [26]:
preds = rf_model.predict(train_values_subset)
f1_score(train_labels, preds, average='micro')

0.9164453484801965

In [27]:
test_values_subset = test_values[features[0:-1]]
test_values_subset = pd.get_dummies(test_values_subset)

In [28]:
predictions = rf_model.predict(test_values_subset)

In [29]:
submission_format = pd.read_csv('submission_format.csv',index_col='building_id')

In [30]:
my_submission = pd.DataFrame(data=predictions, columns=submission_format.columns,
                            index=submission_format.index)

In [31]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,3
890251,3
745817,1
421793,2


In [32]:
my_submission.to_csv('submissionRF.csv')